In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml
from urllib.request import Request, urlopen

In [2]:
WINE_REGION_URL2 = "https://www.tonymappedit.com/top-10-wine-producing-countries/"
WIKI_COUNTRIES = "https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3"

In [3]:
'''scrape wine region data with coresponding wines'''
req = Request(WINE_REGION_URL2 , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

# html_text = requests.get(WINE_REGION_URL).text
soup = BeautifulSoup(webpage, "html.parser")

In [4]:
country1_df = []
countries = soup.find_all("ol")[1]
items = countries.find_all("li")

for country in items:
    country1_df.append(country.text.split(": "))

In [5]:
country2_df = []
countries2 = soup.find_all("ol")[2]
items = countries2.find_all("li")

for country in items:
    country2_df.append(country.text.split(": "))

In [6]:
country_df = country1_df + country2_df
country_df

[['Italy', '54.8'],
 ['France', '49.1'],
 ['Spain', '44.4'],
 ['United States', '23.9'],
 ['Argentina', '14.5'],
 ['Chile', '12.9'],
 ['Australia', '12.9'],
 ['Germany', '9.8'],
 ['South Africa', '9.5'],
 ['China', '9.3'],
 ['Russia', '6.5'],
 ['Portugal', '6.1'],
 ['Romania', '5.1'],
 ['Hungary', '3.6'],
 ['Brazil', '3.1'],
 ['New Zealand', '3.0'],
 ['Austria', '2.8'],
 ['Greece', '2.2'],
 ['Moldova', '1.9'],
 ['Switzerland', '1.1']]

In [7]:
country_production = pd.DataFrame(country_df).rename(columns = {0: "Country", 1: "WineProduction[HCL]"}).set_index("Country")
country_production

,WineProduction[HCL]
Country,
Italy,54.8
France,49.1
Spain,44.4
United States,23.9
Argentina,14.5
Chile,12.9
Australia,12.9
Germany,9.8
South Africa,9.5


In [9]:
'''scrape wine region data with coresponding wines'''
req = Request(WIKI_COUNTRIES , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

# html_text = requests.get(WINE_REGION_URL).text
soup = BeautifulSoup(webpage, "html.parser")

In [10]:
table = soup.find("div", class_="plainlist")
country_codes = table.find_all("span")
countries = table.find_all("a")

In [11]:
country_list = []
for country in countries:
    country_list.append(country.text)

In [12]:
code_list = []
for code in country_codes:
    code_list.append(code.text)

In [13]:
#Province of China was index 229. 
country_df = pd.DataFrame(country_list).drop([11, 229]).reset_index().drop(columns = ["index"])
country_df = country_df.rename(columns = {0: "Country"}).reset_index().set_index("index")

In [14]:
code_df = pd.DataFrame(code_list).drop([11]).reset_index().drop(columns = ["index"])
code_df = code_df.rename(columns = {0: "CODES"}).reset_index().set_index("index")

In [15]:
country_code = pd.merge(country_df, code_df, on="index", how="outer")
country_code

,Country,CODES
index,,
0,Aruba,ABW
1,Afghanistan,AFG
2,Angola,AGO
3,Anguilla,AIA
4,Åland Islands,ALA
...,...,...
243,Samoa,WSM
244,Yemen,YEM
245,South Africa,ZAF


In [16]:
# country_code = pd.concat([country_df, code_df], axis=1).rename(columns = {0: "Country", 1: "Country Codes"})
# country_code

In [17]:
country_code_df = country_code.set_index("Country")
country_code_df

,CODES
Country,
Aruba,ABW
Afghanistan,AFG
Angola,AGO
Anguilla,AIA
Åland Islands,ALA
...,...
Samoa,WSM
Yemen,YEM
South Africa,ZAF


In [18]:
wine_production_HCL = pd.merge(country_production,country_code_df,  how="right", on="Country").rename(columns = {'Alpha-3 code':'CODE'}).fillna(0)

In [19]:
wine_production_HCL

,WineProduction[HCL],CODES
Country,,
Italy,54.8,ITA
France,49.1,FRA
Spain,44.4,ESP
Argentina,14.5,ARG
Chile,12.9,CHL
...,...,...
Wallis and Futuna,0,WLF
Samoa,0,WSM
Yemen,0,YEM


In [20]:
wine_production_HCL.to_csv("wine_production.csv")